In [58]:
import tensorflow as tf
import numpy as np

%matplotlib inline
import pylab

h = tf.Variable(tf.random_uniform([1], 0.0, 2.0))


# Uncomment the following lines to see x.
print(h)

def my_function(h):
  #return 10000.0-((14-2*h)*(10-2*h))
  return 10000.0-(140-96*h+12*h**2)


loss = my_function(h)  # Create an operation that calculates loss.
optimizer = tf.train.GradientDescentOptimizer(0.000001)  # Create an optimizer.
train = optimizer.minimize(loss)  # Create an operation that minimizes loss.
init = tf.initialize_all_variables()  # Create an operation initializes all the variables.

sess = tf.Session()
sess.run(init)
# y_initial_values = sess.run(y)  # Save initial values for plotting later.

# Uncomment the following line to see the initial W and b values.
print(sess.run([h]))

for step in range(15):
    sess.run(train)
    # Uncomment the following two lines to watch training happen real time.
    # if step % 20 == 0:
    print(step, sess.run([h]))
    
print(sess.run([h]))


<tf.Variable 'Variable_3:0' shape=(1,) dtype=float32_ref>
[array([1.3454182], dtype=float32)]
0 [array([1.3453546], dtype=float32)]
1 [array([1.3452909], dtype=float32)]
2 [array([1.3452272], dtype=float32)]
3 [array([1.3451636], dtype=float32)]
4 [array([1.3450999], dtype=float32)]
5 [array([1.3450361], dtype=float32)]
6 [array([1.3449724], dtype=float32)]
7 [array([1.3449086], dtype=float32)]
8 [array([1.3448448], dtype=float32)]
9 [array([1.344781], dtype=float32)]
10 [array([1.3447173], dtype=float32)]
11 [array([1.3446535], dtype=float32)]
12 [array([1.3445897], dtype=float32)]
13 [array([1.3445259], dtype=float32)]
14 [array([1.3444622], dtype=float32)]
[array([1.3444622], dtype=float32)]


In [59]:
H=sess.run([h])
print(H)

A=(14-2*h)*(10-2*h)*h
print(sess.run([A]))

[array([1.3444622], dtype=float32)]
[array([111.181816], dtype=float32)]


In [60]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = x*y+z*((x+2*x*y+3.5)-200)
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 20
    child_num = 5

    space = OrderedDict((
        ('x', (10., 12.)),
        ('y', (5., 7.)),
        ('z', (0., 2.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 11.321748429248265, 5.270997147779575, 1.604354066175953: -45.92881104983422
Eval 11.085929817020881, 6.9485379177777435, 1.1006489852238988: 42.52338757287761
Eval 11.012126147767367, 6.063580530242118, 0.30550381485462386: 50.90442036743052
Eval 10.395971476567937, 6.218558877126279, 0.15854463087689297: 55.64134034105699
Eval 11.84546811621248, 6.310585239202909, 0.35032792162920345: 62.437508731147034
Best [11.321748429248265, 5.270997147779575, 1.604354066175953]

Eval 10.0, 6.929686186630017, 1.4669138038689142: -0.977516082306181
Eval 11.083657722630424, 5.960511089140036, 2.0: -40.51136073488297
Eval 11.681533589889895, 5.585331916229841, 2.0: -43.41072086960533
Eval 10.256957120209625, 5.091646873271497, 0.5384555430333164: 8.18267487914408
Eval 11.32036026748605, 6.09906508579416, 0.4749380512192154: 46.67763587330815
Best [11.681533589889895, 5.585331916229841, 2.0]

Eval 12.0, 7.0, 2.0: 51.0
Eval 12.0, 5.4985730572535, 1.8687783151342139: -32.191984209382596
Eval 12.0,